In [1]:
!docker-compose up -d

Creating network "elasticlab_default" with the default driver
Recreating elasticsearch ... 


In [2]:
!pip3 install elasticsearch
!pip3 install tqdm

    100% |████████████████████████████████| 51kB 1.1MB/s 


In [11]:
### LOAD ARTICLE
import json

articles = []
with open('source/ready_article.json') as f:
    for line in f:
        articles.append(json.loads(line))

In [12]:
### CREATE MAPPING
from elasticsearch import Elasticsearch

es = Elasticsearch(["127.0.0.1:9200"], http_compress=True)

settings = '''
{
	"settings": {
		"number_of_shards": 5,
		"analysis": {
			"analyzer": {
				"search_analyzer": {
                    "filter": ["lowercase", "trim", "apostrophe", "stopwords", "stemmer_id", "stemmer_en"],
					"char_filter": ["html_strip", "url_pattern"],
					"type": "custom",
					"tokenizer": "search_tokenizer"
				}
			},
			"tokenizer": {
				"search_tokenizer": {
					"type": "standard",
					"max_token_length": "45"
				}
			},
			"filter": {
				"stemmer_en": {
					"name": "english",
					"type": "stemmer"
				},
				"stemmer_id": {
					"name": "indonesian",
					"type": "stemmer"
				},
				"stopwords": {
					"type": "stop",
					"stopwords": ["_indonesian_", "_english_"]
				}
			},
			"char_filter": {
				"url_pattern": {
					"pattern": "((https?|ftp|gopher|telnet|file|Unsure|http):((/)|())+[wd:#@%/;$()~_?+-=.&]*)",
					"type": "pattern_replace",
					"replacement": " "
				}
			}
		}
	},
	"mappings": {
		"document": {
			"properties": {
                "id": {
                    "type": "keyword"
                },
                "type": {
                    "type": "keyword"
                },
                "title": {
                    "type": "text",
                    "analyzer": "search_analyzer",
                    "term_vector": "yes"
                },
                "text": {
                    "type": "text",
                    "analyzer": "search_analyzer",
                    "term_vector": "yes"
                },
                "data": {
                    "type": "text",
					"index": false
                },
                "create_time":{
                    "type": "date"
                },
                "update_time":{
                    "type": "date"
                }
			}
		}
	}
}
'''

es.indices.create(index='kurio_doc', ignore=400, body=settings)
es.indices.put_alias(index='kurio_doc',name=)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'kurio_search'}

In [13]:
###INSERT DOCUMENTS
from tqdm import tqdm

articles_bar = tqdm(articles)
for article in articles_bar:
    res = es.index(index="kurio_search", doc_type='article', id=article['id'], body=article)
    articles_bar.set_description("id : {arg1} is {arg2}".format(arg1=article['id'],arg2=res['result']))


id : 11258917 is created: 100%|██████████| 5703/5703 [02:38<00:00, 36.08it/s]
